In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) # makes the notebook fill the whole window

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
import saliency.core as saliency

# Local import
from thumbnail_classification import Classifier

C:\Users\bvano\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
img_size = 32, 32

transform = transforms.Compose([
        transforms.ToTensor(),
        #transforms.Normalize([0.5], [0.5]),
        transforms.Resize(img_size),
    ])

In [4]:
dataset = datasets.ImageFolder("./imgs/", transform = transform)

device = torch.device("cpu")

model = Classifier(device = device, image_size = (3, *img_size), n_classes = len(dataset.classes))
model.load_state_dict(torch.load("./model.pt"))
model = model.to(device)

class_dict = {c:idx for idx, c in enumerate(dataset.classes)}
imgs = torch.stack([x for x,_ in dataset])

FileNotFoundError: [WinError 3] The system cannot find the path specified: './imgs/'

In [5]:
def call_model_function(*args, call_model_args, expected_keys = None):
    output = model(call_model_args["images"])
    output = F.softmax(output, dim = 1)
    if saliency.base.INPUT_OUTPUT_GRADIENTS in expected_keys:
        outputs = output[:, call_model_args["class_idx"]]
        grads = torch.autograd.grad(outputs, call_model_args["images"], grad_outputs = torch.ones_like(outputs))
        grads = torch.movedim(grads[0], 1, 3)
        gradients = grads.detach().numpy()
        return gradients
    else:
        one_hot = torch.zeros_like(output)
        one_hot[:, call_model_args["class_idx"]] = 1
        model.zero_grad()
        output.backward(gradient = one_hot, retain_graph = True)
        return model.grads

In [150]:
img = dataset[0][0].clone().detach()
label = dataset[0][1]

im = img.unsqueeze(dim = 0).to(device).requires_grad_(True)
np_img = im.detach().cpu().numpy().astype(np.float32)

In [151]:
gradient_saliency = saliency.GradientSaliency()

In [152]:
call_model_args = {
    "images" : im,
    "class_idx" : label,
}
vanilla_mask_3d = gradient_saliency.GetMask(im, call_model_function, call_model_args)

RuntimeError: One of the differentiated Tensors does not require grad